In [1]:
#Joint Annotation 만드는 ipynb file

In [2]:
import cv2
import tensorflow as tf
import os
import numpy as np
import network as net
import json
from Training import openpose
import matplotlib.pyplot as plt
from scipy.ndimage.filters import maximum_filter
import math
from collections import Counter, defaultdict
import itertools
import matplotlib as mpl
import time
import ast
from scipy.ndimage.filters import gaussian_filter
import matplotlib

C:\Users\JAEKYU\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Open Pose Class Loading Finish


In [3]:
class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, (np.int_, np.intc, np.intp, np.int8,
            np.int16, np.int32, np.int64, np.uint8,
            np.uint16, np.uint32, np.uint64)):
            return int(obj)
        elif isinstance(obj, (np.float_, np.float16, np.float32, 
            np.float64)):
            return float(obj)
        elif isinstance(obj,(np.ndarray,)): #### This is the fix
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

In [4]:
def make_test_batch(img_path, batch_size):
    num_of_data = len(img_path)
    index = np.arange(0, num_of_data)
    np.random.shuffle(index)
    index = index[:batch_size]
    
    shuffled_img_data = [img_path[i] for i in index]
    
    return np.asarray(shuffled_img_data)

def path_to_image(img_path, batch_size):
    #buffer 선언
    image_data = np.zeros((batch_size, 356, 356, 3), np.uint8)
    
    index = 0
    for img in (img_path):
        image_data[index] = cv2.imread(img)
        index = index + 1;

        
        
    return image_data

def load_test_data(save_path):
    img_path = save_path#"./MPII_Dataset/resized_test_image/"
    file_path = []
    file_list = os.listdir(img_path)
    for i in (file_list):
        file_path.append(img_path + i)
    return file_path

def padRightDownCorner(img, stride, padValue):
    h = img.shape[0]
    w = img.shape[1]

    pad = 4 * [None]
    pad[0] = 0 # up
    pad[1] = 0 # left
    pad[2] = 0 if (h%stride==0) else stride - (h % stride) # down
    pad[3] = 0 if (w%stride==0) else stride - (w % stride) # right

    img_padded = img
    pad_up = np.tile(img_padded[0:1,:,:]*0 + padValue, (pad[0], 1, 1))
    img_padded = np.concatenate((pad_up, img_padded), axis=0)
    pad_left = np.tile(img_padded[:,0:1,:]*0 + padValue, (1, pad[1], 1))
    img_padded = np.concatenate((pad_left, img_padded), axis=1)
    pad_down = np.tile(img_padded[-2:-1,:,:]*0 + padValue, (pad[2], 1, 1))
    img_padded = np.concatenate((img_padded, pad_down), axis=0)
    pad_right = np.tile(img_padded[:,-2:-1,:]*0 + padValue, (1, pad[3], 1))
    img_padded = np.concatenate((img_padded, pad_right), axis=1)

    return img_padded, pad

In [5]:
#f = open("./Video_annotation/1.txt", 'w')

def demo_view(oriImg, input_heatmap, input_paf):
    nms_joint = []
    #input_heatmap = np.squeeze(input_heatmap)
    heatmap_avg = np.zeros((oriImg.shape[0], oriImg.shape[1], 17))
    paf_avg = np.zeros((oriImg.shape[0], oriImg.shape[1], 34))

    
    heatmap_avg = heatmap_avg + input_heatmap
    paf_avg = paf_avg + input_paf
    all_peaks = []
    peak_counter = 0

    for part in range(17-1):
        map_ori = heatmap_avg[:,:,part]
        map = gaussian_filter(map_ori, sigma=3)

        map_left = np.zeros(map.shape)
        #print(np.shape(map_left))
        map_left[1:,:] = map[:-1,:]
        map_right = np.zeros(map.shape)
        map_right[:-1,:] = map[1:,:]
        map_up = np.zeros(map.shape)
        map_up[:,1:] = map[:,:-1]
        map_down = np.zeros(map.shape)
        map_down[:,:-1] = map[:,1:]

        peaks_binary = np.logical_and.reduce((map>=map_left, map>=map_right, map>=map_up,\
                                              map>=map_down, map > 0.1))
        peaks = list(zip(np.nonzero(peaks_binary)[1], np.nonzero(peaks_binary)[0])) # note reverse
        peaks_with_score = [x + (map_ori[x[1],x[0]],) for x in peaks]
        id = range(peak_counter, peak_counter + len(peaks))
        peaks_with_score_and_id = [peaks_with_score[i] + (id[i],) for i in range(len(id))]

        all_peaks.append(peaks_with_score_and_id)
        peak_counter += len(peaks)
    """connection_all = []
    special_k = []
    mid_num = 10###########################################################################################
    #paf_avg = resized_vectormap
    for k in range(len(mapIdx)):
        score_mid = paf_avg[:,:,[x-17 for x in mapIdx[k]]]
        #print("score_mid : ",np.shape(score_mid))
        candA = all_peaks[limbSeq[k][0]]#all_peaks[limbSeq[k][0]-1]#157
        candB = all_peaks[limbSeq[k][1]]#all_peaks[limbSeq[k][1]-1]#246
        #candA : [(182, 68, 0.8127860426902771, 9)]
        #candB :[(173, 92, 0.8721999526023865, 8)]
        #print("candA, candB : ",candA, candB)
        nA = len(candA)
        nB = len(candB)
        #print("nA, nB : ", nA, nB)
        indexA, indexB = limbSeq[k]
        #print("indexA, indexB : ", indexA, indexB)
        if(nA != 0 and nB != 0):
            connection_candidate = []
            for i in range(nA):
                for j in range(nB):
                    vec = np.subtract(candB[j][:2], candA[i][:2])
                    norm = math.sqrt(vec[0]*vec[0] + vec[1]*vec[1])
                    # failure case when 2 body parts overlaps
                    if norm == 0:
                        continue
                    vec = np.divide(vec, norm)

                    startend = list(zip(np.linspace(candA[i][0], candB[j][0], num=mid_num), \
                                   np.linspace(candA[i][1], candB[j][1], num=mid_num)))

                    vec_x = np.array([score_mid[int(round(startend[I][1])), int(round(startend[I][0])), 0] \
                                      for I in range(len(startend))])

                    vec_y = np.array([score_mid[int(round(startend[I][1])), int(round(startend[I][0])), 1] \
                                      for I in range(len(startend))])

                    score_midpts = np.multiply(vec_x, vec[0]) + np.multiply(vec_y, vec[1])
                    score_with_dist_prior = sum(score_midpts)/len(score_midpts) + min(0.5*oriImg.shape[0]/norm-1, 0)
                    #print("score_midpts : ", score_midpts)
                    criterion1 = len(np.nonzero(score_midpts > 0.0001)[0]) > 0.00005 * len(score_midpts)
                    #print("score_with_dist_prior : ", score_with_dist_prior)
                    criterion2 = score_with_dist_prior > 0.1
                    #print("criterion1, criterion2 : ", criterion1, criterion2)
                    if criterion1 and criterion2:
                        #print("1")
                        connection_candidate.append([i, j, score_with_dist_prior, score_with_dist_prior+candA[i][2]+candB[j][2]])
            #print("connection_candidate : ", connection_candidate)
            connection_candidate = sorted(connection_candidate, key=lambda x: x[2], reverse=True)
            connection = np.zeros((0,5))
            #print("connection_candidate : ", connection_candidate)
            for c in range(len(connection_candidate)):
                i,j,s = connection_candidate[c][0:3]
                if(i not in connection[:,3] and j not in connection[:,4]):
                    connection = np.vstack([connection, [candA[i][3], candB[j][3], s, i, j]])
                    if(len(connection) >= min(nA, nB)):
                        break
            #print("connection : ", connection)
            connection_all.append(connection)
        else:
            special_k.append(k)
            connection_all.append([])
    subset = -1 * np.ones((0, 20))######################################################
    #print("subset1 : ", subset)
    candidate = np.array([item for sublist in all_peaks for item in sublist])
    #print("candidate : ", candidate)
    #print("all_peaks : ", all_peaks)
    for k in range(len(mapIdx)):
        #print("--------------------------------------------")
        #print("k : ", k)
        #print("special_k : ", special_k)
        if k not in special_k:
            #print("k is not in special_k : ",k)
            partAs = connection_all[k][:,0]
            partBs = connection_all[k][:,1]
            indexA, indexB = np.array(limbSeq[k]) - 1

            #print("connection_all : ",connection_all)
            #print("connection_all[k] : ",connection_all[k])
            for i in range(len(connection_all[k])): #= 1:size(temp,1)
                #print("hello")
                found = 0
                subset_idx = [-1, -1]
                #print("subset's length :", len(subset))
                for j in range(len(subset)): #1:size(subset,1):
                    #print("subset3 : ", subset)
                    if subset[j][indexA] == partAs[i] or subset[j][indexB] == partBs[i]:
                        subset_idx[found] = j
                        found += 1
                #print("found : ",found)
                if found == 1:
                    j = subset_idx[0]
                    if(subset[j][indexB] != partBs[i]):
                        subset[j][indexB] = partBs[i]
                        subset[j][-1] += 1
                        subset[j][-2] += candidate[partBs[i].astype(int), 2] + connection_all[k][i][2]
                elif found == 2: # if found 2 and disjoint, merge them
                    j1, j2 = subset_idx
                    #print ("found = 2")
                    membership = ((subset[j1]>=0).astype(int) + (subset[j2]>=0).astype(int))[:-2]
                    if len(np.nonzero(membership == 2)[0]) == 0: #merge
                        subset[j1][:-2] += (subset[j2][:-2] + 1)
                        subset[j1][-2:] += subset[j2][-2:]
                        subset[j1][-2] += connection_all[k][i][2]
                        subset = np.delete(subset, j2, 0)
                    else: # as like found == 1
                        subset[j1][indexB] = partBs[i]
                        subset[j1][-1] += 1
                        subset[j1][-2] += candidate[partBs[i].astype(int), 2] + connection_all[k][i][2]

                # if find no partA in the subset, create a new subset
                elif not found and k < 17:
                    row = -1 * np.ones(20)
                    row[indexA] = partAs[i]
                    row[indexB] = partBs[i]
                    row[-1] = 2
                    row[-2] = sum(candidate[connection_all[k][i,:2].astype(int), 2]) + connection_all[k][i][2]
                    subset = np.vstack([subset, row])
    #print("end~",subset)
    # delete some rows of subset which has few parts occur
    deleteIdx = [];
    #print(subset)
    for i in range(len(subset)):
        #print(subset[i][-1])
        if subset[i][-1] < 1 or subset[i][-2]/subset[i][-1] < 0.3:
            deleteIdx.append(i)
            #print(i)
    subset = np.delete(subset, deleteIdx, axis=0)
    #print(subset)
    # visualize

    cmap = matplotlib.cm.get_cmap('hsv')

    canvas = oriImg#cv2.imread(oriImg) # B,G,R order
    #canvas = canvas[:, :, (2,1,0)]"""
    #print(all_peaks)########################################################################################
    nms_joint.append(all_peaks)
    #all_peaks_file = ",".join(str(x) for x in all_peaks)
    #f.write(all_peaks_file)
    #f.write("\n")
    
    """for i in range(16):
        rgba = np.array(cmap(1 - i/16. - 1./32))
        rgba[0:3] *= 255

        for j in range(len(all_peaks[i])):
            #print((all_peaks[i][j]))
            #x = all_peaks[i][j][3]
            #y = all_peaks[i][j][1]
            cv2.circle(canvas, all_peaks[i][j][0:2], 2, colors[i], thickness=-1)
            #cv2.circle(canvas, (x, y), 1, colors[i], thickness=-1)

    to_plot = cv2.addWeighted(oriImg, 0.3, canvas, 0.7, 0)


    # visualize 2# visual 
    stickwidth = 1

    for i in range(15):
        for n in range(len(subset)):
            index = subset[n][np.array(limbSeq[i])-1]
            #print("index : ", index)
            if -1 in index:
                continue
            #print("hello")
            cur_canvas = canvas.copy()
            Y = candidate[index.astype(int), 0]
            X = candidate[index.astype(int), 1]
            mX = np.mean(X)
            mY = np.mean(Y)
            length = ((X[0] - X[1]) ** 2 + (Y[0] - Y[1]) ** 2) ** 0.5
            angle = math.degrees(math.atan2(X[0] - X[1], Y[0] - Y[1]))
            polygon = cv2.ellipse2Poly((int(mY),int(mX)), (int(length/2), stickwidth), int(angle), 0, 360, 1)
            cv2.fillConvexPoly(cur_canvas, polygon, colors[i])
            canvas = cv2.addWeighted(canvas, 0.4, cur_canvas, 0.6, 0)"""
    #return canvas[:,:,[2,1,0]], nms_joint
    return nms_joint

In [6]:
limbSeq = [(9, 8), (8, 13), (8, 12), (8, 7), (12, 11), (11, 10), (13, 14), (14, 15), \
           (7, 6), (6, 3),  (3, 4),  (4, 5), (6, 2),   (2, 1),   (1, 0)]#16개 조인트, 15개 limb

mapIdx = [(16, 17), (18, 19), (20, 21), (22, 23), (28, 29), (30, 31), (24, 25), (26, 27), (32, 33), \
          (34, 35), (38, 39), (40, 41), (36, 37), (42, 43), (44, 45)]#x, y vectormap Pairs 24,25

colors = [[255, 0, 0], [255, 85, 0], [255, 170, 0], [255, 255, 0], [170, 255, 0], [85, 255, 0], [0, 255, 0], \
          [0, 255, 85], [0, 255, 170], [0, 255, 255], [0, 170, 255], [0, 85, 255], [0, 0, 255], [85, 0, 255], \
          [170, 0, 255], [255, 0, 255], [255, 0, 170], [255, 0, 85]]
img_width = 112
video_data_path = "./video_with_abstacle/TrainingSet_2/video_another_30/"
video_list = os.listdir(video_data_path)
video_filename_list = os.listdir(video_data_path)
for i, video in enumerate(video_list):
    video_list[i] = video_data_path + video
with tf.Session() as sess:
#with tf.Session(config=tf.ConfigProto(device_count={'GPU': 0})) as sess:
    obj = openpose(batch_size=1, sess = sess)
    sess.run(tf.global_variables_initializer())
    SAVE_PATH = "C:/Users/JAEKYU/Documents/Jupyter Notebook/Open_Pose/Weight/Weight.ckpt"
    saver = tf.train.Saver()
    saver.restore(sess, SAVE_PATH)
    for index, video in enumerate(video_list):
        #cap = cv2.VideoCapture(0)
        print(video)
        cap = cv2.VideoCapture(video)

        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        #out = cv2.VideoWriter('output2.avi',fourcc, 30.0, (356,356))
        #while(True):
        sum_nms_joint = []
        #print("frame start")
        for index_sub in range(60):
            #print("caption read")
            ret, img_color = cap.read()
            if(ret == True):
                img_color = cv2.resize(img_color, (356, 356))
                heatmap, vectormap = obj.demo_test([img_color])
                
                heatmap = np.squeeze(heatmap)
                vectormap = np.squeeze(vectormap)
                resized_heatmap = np.zeros((17, 356, 356))
                resized_vectormap = np.zeros((34, 356, 356))
                
                heatmap = np.transpose(heatmap, (2,0, 1))
                vectormap = np.transpose(vectormap, (2,0, 1))
                
                for k in range(17):
                    resized_heatmap[k] = cv2.resize(heatmap[k], (356, 356))
                for l in range(34):
                    resized_vectormap[l] = cv2.resize(vectormap[l], (356, 356))

                resized_heatmap = np.transpose(resized_heatmap, (1,2, 0))
                resized_vectormap = np.transpose(resized_vectormap, (1,2, 0))

                #output, nms_joint = demo_view(img_color, resized_heatmap, resized_vectormap)
                nms_joint = demo_view(img_color, resized_heatmap, resized_vectormap)

                sum_nms_joint.append(nms_joint)

            if ret == False:
                continue
            #output = output[...,::-1]
            #cv2.imshow('bgr', output)
            #27 -> esc
            if cv2.waitKey(1) & 0xFF == 27:
                break
        json_nms_joint = dict()
        with open("./video_with_abstacle/TrainingSet_2/video_another_30_annotaion/"+video_filename_list[index][0:-4]+'.json', 'w') as f:
            #print(len(sum_nms_joint))
            for i in range(len(sum_nms_joint)):
                for j in range(16):

                    if(len(sum_nms_joint[i][0][j]) == 0):
                        pass
                    else:
                        #print(j, nms_joint[i][j][0][0:2])
                        #print(type())
                        json_nms_joint['%d'%j] = sum_nms_joint[i][0][j][0][0:2]
                #json_nms_joint['%d'%j] = sum_nms_joint[i][j][0][0:2]
                #print(json_nms_joint)
                json_nms_joint
                #json.
                dumped = json.dumps(json_nms_joint, cls=NumpyEncoder)

                json.dump(dumped, f)
                f.write('\n')

                json_nms_joint.clear()
                #print("------------------------------------")
            f.close()

        cap.release()
        #out.release()
        cv2.destroyAllWindows()


test data load start
test data load finish
open pose init complete
INFO:tensorflow:Restoring parameters from C:/Users/JAEKYU/Documents/Jupyter Notebook/Open_Pose/Weight/Weight.ckpt
./video_with_abstacle/TrainingSet_2/video_another_30/101.avi
./video_with_abstacle/TrainingSet_2/video_another_30/102.avi
./video_with_abstacle/TrainingSet_2/video_another_30/103.avi
./video_with_abstacle/TrainingSet_2/video_another_30/104.avi
./video_with_abstacle/TrainingSet_2/video_another_30/105.avi
./video_with_abstacle/TrainingSet_2/video_another_30/106.avi
./video_with_abstacle/TrainingSet_2/video_another_30/108.avi
./video_with_abstacle/TrainingSet_2/video_another_30/115.avi
./video_with_abstacle/TrainingSet_2/video_another_30/117.avi
./video_with_abstacle/TrainingSet_2/video_another_30/120.avi
./video_with_abstacle/TrainingSet_2/video_another_30/124.avi
./video_with_abstacle/TrainingSet_2/video_another_30/125.avi
./video_with_abstacle/TrainingSet_2/video_another_30/128.avi
./video_with_abstacle/Trai

In [7]:
sum_nms_joint[0][0][0][0]#[image][0][part][몇 번째 사람][0:2]

IndexError: list index out of range

In [ ]:
sum_nms_joint[0]

In [ ]:
json_nms_joint = dict()
with open('filename.json', 'w') as f:
    for i in range(len(nms_joint)):
        for j in range(16):
            if(len(nms_joint[i][j]) == 0):
                pass
            else:
                #print(j, nms_joint[i][j][0][0:2])
                json_nms_joint['%d'%j] = nms_joint[i][j][0][0:2]
        
        print(json_nms_joint)
        dumped = json.dumps(json_nms_joint, cls=NumpyEncoder)

        json.dump(dumped, f)
        f.write('\n')

        json_nms_joint.clear()
        print("------------------------------------")
        
    

In [9]:
v_list = os.listdir("./video_with_abstacle/TrainingSet_2/video_another_30/")

In [10]:
for i, d in enumerate (v_list):
    v_list[i] = "./video_with_abstacle/TrainingSet_2/video_another_30/" + d

In [11]:
for i in v_list:
    cap = cv2.VideoCapture(i)
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if(length != 30):
        print(i)

In [119]:
cap = cv2.VideoCapture(video_path)
    video_data = []
    i = 0
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    while(i != length):
        ret, frame = cap.read()
        #gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        frame = cv2.resize(frame, (img_width, img_width))
        video_data.append(frame)
        i = i + 1
    return joint_data, video_data